In [ ]:
import os
import torch
import requests
from PIL import Image
from transformers import AutoProcessor, OmDetTurboForObjectDetection

processor = AutoProcessor.from_pretrained("omlab/omdet-turbo-swin-tiny-hf",use_fast=True)
model = OmDetTurboForObjectDetection.from_pretrained("omlab/omdet-turbo-swin-tiny-hf")

In [ ]:
def load_images_from_folder(dataset_path):
    images = []

    # Iterate through all files in the directory
    for filename in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, filename)

        # Check if it's a file and has a valid image extension
        if os.path.isfile(file_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                image = Image.open(file_path).convert("RGB")
                images.append(image)
            except Exception as e:
                print(f"Error opening {file_path}: {e}")

    return images

dataset_path = "/content/dataset"
images = load_images_from_folder(dataset_path)

classes = ["car", "people"]
task = "Detect car and people."

In [ ]:
batch_size = 30
results = []

for i in range(0, len(images), batch_size):
    batch_imgs = images[i:i+batch_size]
    batch_texts = [classes] * len(batch_imgs)
    batch_tasks = [task] * len(batch_imgs)

    inputs = processor(
        images=batch_imgs,
        text=batch_texts,
        task=batch_tasks,
        return_tensors="pt",
        padding=True
    )

    with torch.no_grad():
        batch_output = model(**inputs)

    batch_results = processor.post_process_grounded_object_detection(
        batch_output,
        classes=[classes] * len(batch_imgs),
        target_sizes=[img.size[::-1] for img in batch_imgs],
        score_threshold=0.2,
        nms_threshold=0.3,
    )

    # Aggiungi i risultati del batch direttamente in results
    results.extend(batch_results)

Unused or unrecognized kwargs: masks_path, annotations.
<ipython-input-6-d4c6576cb7e2>:20: FutureWarning: `score_threshold` is deprecated and will be removed in version 4.51.0 for `OmDetTurboProcessor.post_process_grounded_object_detection`. Use `threshold` instead.
  batch_results = processor.post_process_grounded_object_detection(


In [ ]:
for i, result in enumerate(results):
    for score, class_name, box in zip(
        result["scores"], result["classes"], result["boxes"]
    ):
        box = [round(i, 1) for i in box.tolist()]
        print(
            f"Detected {class_name} with confidence "
            f"{round(score.item(), 2)} at location {box} in image {i}"
        )

In [ ]:
!rm -rf /content/dataset